In [1]:
import azureml
from azureml.core import Workspace, Dataset
from azureml.core import Experiment
from azureml.core.run import Run 
import sys

import tensorflow as tf 
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
#from tensorflow.keras.models import load_model
import numpy as np
from pathlib import Path
import glob2 as glob
import random
import pickle

from cgmml.common.evaluation.eval_utilities import download_model

REPO_DIR = Path(os.getcwd()).parents[2].absolute()

In [ ]:
! echo $PYTHONPATH
print(sys.path)

In [ ]:
print(REPO_DIR)

In [ ]:
print(tf.__version__)

### Connect to azureml workspace

In [ ]:
workspace = Workspace.from_config()
workspace

### Specify experiment and run_id to download model

In [4]:
experiment_name = "q3-depthmap-plaincnn-height-95k"
run_id = "q3-depthmap-plaincnn-height-95k_1629821224_3ce63344"  # Run 2
input_location = 'outputs/best_model.ckpt'
#input_location = 'outputs' # throws NameError: outputs's path extension not supported - from eval_utilities.py
name = 'best_model.ckpt'
#output_location = (REPO_DIR / 'data' / run_id)
output_location = (REPO_DIR / 'data' / experiment_name)

In [ ]:
download_model(workspace=workspace, 
              experiment_name=experiment_name,
              run_id=run_id,
              input_location=input_location,
              output_location=output_location)

In [ ]:
print(output_location)

In [5]:
#model_file_path = (output_location / 'outputs' / 'best_model.ckpt' / 'saved_model.pb')
model_file_path = (output_location / 'outputs' / 'best_model.ckpt')
print(model_file_path)

/Users/jasmin/CGM_local/cgm-ml/data/q3-depthmap-plaincnn-height-95k/outputs/best_model.ckpt


In [ ]:
#ckpt = tf.train.Checkpoint()
#manager = tf.train.CheckpointManager(ckpt, model_file_path, max_to_keep=3)

In [ ]:
#ckpt.restore(manager.latest_checkpoint)
  #if manager.latest_checkpoint:
#print("Restored from {}".format(manager.latest_checkpoint))


In [23]:
model = tf.keras.models.load_model(model_file_path, compile=False)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 240, 180, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 240, 180, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 120, 90, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 90, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 120, 90, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 45, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 60, 45, 128)       7

In [24]:
print(type(model))

<class 'tensorflow.python.keras.engine.sequential.Sequential'>


### Consume dataset and find some examples

In [ ]:
# ALREADY DOWNLOADED
#dataset = Dataset.get_by_name(workspace, name='anon-depthmap-mini')
#dataset.download(target_path='./dataset', overwrite=False)

Get a random examples out

In [ ]:
#def get_depthmap_files(paths):
#    pickle_paths = []
#    for path in paths:
#        pickle_paths.extend(glob.glob(os.path.join(path, "*.p")))
#    return pickle_paths

In [8]:
dataset_path = "./dataset/scans"
qrcode_paths = glob.glob(os.path.join(dataset_path, "*"))
print('qrcode_paths: %d', len(qrcode_paths))
print(qrcode_paths[0])
print(qrcode_paths[1])

qrcode_paths: %d 1745
./dataset/scans/1585361581-riv92cc0l8
./dataset/scans/1585359783-15m68f6s91


In [ ]:
#qrcode_paths_activation = random.choice(qrcode_paths)
#qrcode_paths_activation = [qrcode_paths_activation]
#print(len(qrcode_paths_activation))
#print(qrcode_paths_activation)

In [ ]:
! ls ./dataset/scans/1585273961-7dw2qedbor/100

In [ ]:
#scantype_path = "./dataset/scans/1585273961-7dw2qedbor/100/"
#print(scantype_path)
#path_activate = get_depthmap_files(scantype_path)
#print(len(path_activate))

In [9]:
scan_paths = ["./dataset/scans/1585273961-7dw2qedbor/100/pc_1585273961-7dw2qedbor_1592533886956_100_000.p", 
              "./dataset/scans/1585273961-7dw2qedbor/100/pc_1585273961-7dw2qedbor_1592533886956_100_014.p"]

In [10]:
def calculate_pickle(path):
    def find_max(path):
        depthmap, targets = pickle.load(open(path.numpy(), "rb"))
        depthmap = preprocess_depthmap(depthmap)
        depthmax = tf.math.reduce_max(depthmap)
        return depthmax
    sample = []
    depthmap_maximum = tf.py_function(find_max, [path], [tf.float32])
    sample.append(depthmap_maximum)
    return sample

def preprocess_depthmap(depthmap):
    # TODO here be more code.
    return depthmap.astype("float32")

def preprocess_targets(targets, targets_indices):
    if targets_indices is not None:
        targets = targets[targets_indices]
    return targets.astype("float32")

def get_max(dataset):
    global_max = []
    for value in dataset:
        max_value = value[0].numpy()[0]
        global_max.append(max_value)
    max_array = np.array(global_max)
    max_value = max_array.max()
    return max_value

def tf_load_pickle(path, max_value):
    def py_load_pickle(path, max_value):
        depthmap, targets = pickle.load(open(path.numpy(), "rb"))
        depthmap = preprocess_depthmap(depthmap)
        depthmap = depthmap / max_value
        depthmap = tf.image.resize(depthmap, (image_target_height, image_target_width))
        targets = preprocess_targets(targets, targets_indices)
        return depthmap, targets

    depthmap, targets = tf.py_function(py_load_pickle, [path, max_value], [tf.float32, tf.float32])
    depthmap.set_shape((image_target_height, image_target_width, 1))
    targets.set_shape((len(targets_indices,)))
    return depthmap, targets

In [11]:
# PLEASE REMOVE THIS, THIS IS BAD
image_target_width = 180
image_target_height = 240
targets_indices = [0]

In [12]:
# understand better what happens here
paths = scan_paths
dataset_current = tf.data.Dataset.from_tensor_slices(paths)
dataset_current = dataset_current.map(lambda path: calculate_pickle(path))
dataset_max = get_max(dataset_current)
#logger.info('Maximum value of activation dataset: %.2f', dataset_max)
dataset = tf.data.Dataset.from_tensor_slices(paths)
dataset = dataset.map(lambda path: tf_load_pickle(path, dataset_max))
dataset = dataset.cache()
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
dataset_activation = dataset
del dataset

2021-09-03 16:10:03.415005: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


In [30]:
print(len(dataset_activation))
print(type(dataset_activation))

2
<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


### GradCAM class

In [31]:
class GradCAM:
    def __init__(self, model, layerName):
        self.model = model
        self.layerName = layerName

        self.gradModel = Model(inputs=[self.model.inputs],
                               outputs=[self.model.get_layer(self.layerName).output, model.output])

    def compute_heatmap(self, image, classIdx, eps=1e-8):

        with tf.GradientTape() as tape:
            #tape.watch(self.gradModel.get_layer(self.layerName).output)
            tape.watch(tf.convert_to_tensor(self.gradModel.get_layer(self.layerName).output))
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = self.gradModel(inputs)
            if len(predictions) == 1:
                loss = predictions[0]
            else:
                loss = predictions[:, classIdx]

        grads = tape.gradient(loss, convOutputs)

        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads

        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]

        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))

        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("float32")
        return heatmap

### Instantiate GradCAM class

last convolutional layer: conv2d_11 (Conv2D)           (None, 7, 5, 256)         590080

In [25]:
layer_name = 'conv2d_11'
cam = GradCAM(model, layer_name)

In [32]:
for data in dataset_activation:
    image = data[0]
    image = np.expand_dims(image, 0)
    pred = model.predict(image)
    classIDx = np.argmax(pred[0])
    print(type(image))
    print(type(classIDx))
    
    ## Compute Heatmap
    heatmap = cam.compute_heatmap(image, classIDx)
    #image = image.reshape(image.shape[1:])
    #image = image * 255
    #image = image.astype(np.uint8)

<class 'numpy.ndarray'>
<class 'numpy.int64'>


ValueError: Passed in object of type <class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>, not tf.Tensor